<a href="https://colab.research.google.com/github/varshathejes/Churn-Prediction-Neural-Network/blob/main/Churn_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
#Importing the dataset
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

# to transform the categorical values to numerical labels. 
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])

#to encode the categorical variables as binary vectors. 
#The remainder parameter is set to 'passthrough' to keep the remaining columns of the input data unchanged.
ct = ColumnTransformer([('one_hot_encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = ct.fit_transform(X)

# Remove the first column to avoid the dummy variable trap
X = X[:, 1:]

In [ ]:

# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
#for building and training neural networks.
import keras
from keras.models import Sequential # Required to initialise the Neural network
from keras.layers import Dense # Required to build the layers of the ANN


In [ ]:
# Initialising the ANN
classifier = Sequential() # classifier represents the neural network here
# units = number of nodes in the hidden layer(Avg of no. of nodes in input and output layers)

classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
#returns the input value if positive and returns zero otherwise. The mathematical formula for the ReLU activation function is:
#f(x) = max(0, x)    input_dim parameter is the number of features.

classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))

classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
# sigmoid= to obtain a probability output between 0 and 1.

In [ ]:

# Compiling the ANN(Applying Stochastic Gradient Descent to whole  neural network)
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
# 'adam' = Stochastic Gradient Descent
#binary_crossentropy= loss function for binary classification problems.
#metrics = accuracy
# Fitting the ANN to the Training set
classifier.fit(X_train, y_train, batch_size = 10, epochs = 100)

# batch_size means update weights after 10 sets of observations

Epoch 1/100
800/800 [==============================] - 2s 1ms/step - loss: 0.4950 - accuracy: 0.7951
Epoch 2/100
800/800 [==============================] - 1s 1ms/step - loss: 0.4286 - accuracy: 0.7960
Epoch 3/100
800/800 [==============================] - 1s 1ms/step - loss: 0.4235 - accuracy: 0.7960
Epoch 4/100
800/800 [==============================] - 1s 1ms/step - loss: 0.4191 - accuracy: 0.8120
Epoch 5/100
800/800 [==============================] - 1s 2ms/step - loss: 0.4167 - accuracy: 0.8254
Epoch 6/100
800/800 [==============================] - 1s 1ms/step - loss: 0.4145 - accuracy: 0.8278
Epoch 7/100
800/800 [==============================] - 1s 1ms/step - loss: 0.4132 - accuracy: 0.8300
Epoch 8/100
800/800 [==============================] - 1s 1ms/step - loss: 0.4120 - accuracy: 0.8310
Epoch 9/100
800/800 [==============================] - 1s 1ms/step - loss: 0.4109 - accuracy: 0.8317
Epoch 10/100
800/800 [==============================] - 1s 1ms/step - loss: 0.4099 - accura

In [ ]:
# Part 3 - Making predictions and evaluating the model

# Predicting the Test set results
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

# Predicting a single new observation
"""Predict if the customer with the following informations will leave the bank:
Geography: France
Credit Score: 600
Gender: Male
Age: 40
Tenure: 3
Balance: 60000
Number of Products: 2
Has Credit Card: Yes
Is Active Member: Yes
Estimated Salary: 50000"""
new_prediction = classifier.predict(sc.transform(np.array([[0.0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])))
new_prediction = (new_prediction > 0.5)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test,y_pred)


1/1 [==============================] - 0s 23ms/step
[[False]]


In [ ]:
# Part 4 - Evaluating, Improving and Tuning the ANN

# Evaluating the ANN
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense
def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier
classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 100)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs = -1)
mean = accuracies.mean()
variance = accuracies.std()



<ipython-input-23-accd8f8bee13>:15: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 100)


In [ ]:
# Improving the ANN
# Dropout Regularization to reduce overfitting if needed
# Lookout for dropout layers below the Dense Layers above.

# Tuning the ANN
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
def build_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier
classifier = KerasClassifier(build_fn = build_classifier)
parameters = {'batch_size': [25, 32],
              'epochs': [100, 500],
              'optimizer': ['adam', 'rmsprop']}

# Always use 'rmsprop' for Recurrent Neural Networks

<ipython-input-24-05157ec92922>:17: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  classifier = KerasClassifier(build_fn = build_classifier)


In [ ]:
grid_search = GridSearchCV(estimator = classifier,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 10)
grid_search = grid_search.fit(X_train, y_train)
best_parameters = grid_search.best_params_
best_accuracy = grid_search.best_score_

# Note : The above process, i.e., tuning the ANN will take quite some time.

Streaming output truncated to the last 5000 lines.
288/288 [==============================] - 0s 1ms/step - loss: 0.3947 - accuracy: 0.8371
Epoch 79/100
288/288 [==============================] - 0s 1ms/step - loss: 0.3948 - accuracy: 0.8385
Epoch 80/100
288/288 [==============================] - 0s 1ms/step - loss: 0.3943 - accuracy: 0.8376
Epoch 81/100
288/288 [==============================] - 0s 1ms/step - loss: 0.3944 - accuracy: 0.8364
Epoch 82/100
288/288 [==============================] - 0s 1ms/step - loss: 0.3944 - accuracy: 0.8371
Epoch 83/100
288/288 [==============================] - 0s 1ms/step - loss: 0.3944 - accuracy: 0.8363
Epoch 84/100
288/288 [==============================] - 0s 1ms/step - loss: 0.3942 - accuracy: 0.8371
Epoch 85/100
288/288 [==============================] - 0s 1ms/step - loss: 0.3943 - accuracy: 0.8389
Epoch 86/100
288/288 [==============================] - 0s 2ms/step - loss: 0.3936 - accuracy: 0.8374
Epoch 87/100
288/288 [======================